In [1]:
import folium
import os
import csv
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import shapefile
import pickle
import random
#import gams_magic
import networkx as nx

def rgb_to_hex(r, g, b):
    return ('#{:X}{:X}{:X}').format(r, g, b)

pathFile = 'trajectories_local_50_w09_route_route.pk'

# read link data
road_file_path = os.path.join('..','..','road_file.csv')
road_file = pd.read_csv(road_file_path,names=["RDWY_LINK_ID","REF_SITE_FROM_ID","REF_SITE_TO_ID"])

# read road shape (shape file already converted using CRS in qgis)
shape_path = os.path.join('..',"..","transportation_data","Middleton_Cross_Plains","Features","Middleton_Road_New.shp")
shape = shapefile.Reader(shape_path)

# read crash shape
crash_path = os.path.join('..',"..","transportation_data","Middleton_Cross_Plains","Features","Crash_data_combined_2017_2020.shp")
crash = shapefile.Reader(crash_path)

# load reference point coordinate
reference_coordinate = None
reference_path = os.path.join('..',"..","reference_coordinate")




In [5]:
############# create road shape dictionary ############
road_vector = {}
for sp in shape.shapeRecords():
    # different road shape can share the same road id
    road_id = sp.record[43]
    

    x = [i[0] for i in sp.shape.points[:]]
    y = [i[1] for i in sp.shape.points[:]]
    
    segment = []
    for i in range(len(x)):
        segment.append((y[i],x[i]))
        
    if road_id in road_vector:
        road_vector[road_id].append(segment)
    else:
        road_vector[road_id] = [segment]


with open(reference_path, 'rb') as f:
    reference_coordinate = pickle.load(f)

# change id to road
idToRoad =  []
with open("../../turning_network_linkID_updated.csv",'r') as f:
    idToRoad = csv.reader(f)
    idToRoad = list(idToRoad)
    
idToRoadDict = {}
roadToIdDict ={}
for element in idToRoad:
    idToRoadDict[element[0]] = (element[1],element[2])
    if "intersection" not in element[0]:
        roadToIdDict[element[1].split('_')[0],element[2].split('_')[0]] = element[0]

with open(pathFile, 'rb') as f:
    path = pickle.load(f)



In [8]:
group_optimal_path = []
for r in path['l1']:
    if 'intersection' not in r:
        group_optimal_path.append(r)
    

In [12]:
group_optimal_path = [group_optimal_path]

In [14]:
clusterNum = len(group_optimal_path)

optimal_roadIdList = []
for g in range(clusterNum):
    example_coordinate = []
    for ind in range(len(group_optimal_path[g])):
            #print((example_trip[ind],example_trip[ind+1]))
            example_coordinate += road_vector[int(group_optimal_path[g][ind])]    
    optimal_roadIdList.append(example_coordinate)

## plot out the optimal path and save it
colors = [rgb_to_hex(128, 128, 128),'red','green','blue','black']
for g in range(clusterNum):
    m = folium.Map(location=[43.0819, -89.5579])
    for loc in optimal_roadIdList[g]:
        folium.PolyLine(loc,
                    #color=rgb_to_hex(random.randint(1,256), random.randint(1,256), random.randint(1,256)),
                    color=colors[g],
                    weight=5.5-1*g,
                    opacity=0.8).add_to(m)

    
    m.save('w09_example.html')


In [11]:
road_vector

{4840779: [[(43.11424116746358, -89.65403976664703),
   (43.114208028684324, -89.65376263717621),
   (43.11410291021964, -89.65283988490333)]],
 4840771: [[(43.114365875964424, -89.65519123502214),
   (43.114360330805766, -89.65510818799652),
   (43.11434207705771, -89.65490481729512),
   (43.114332061226776, -89.6548025224123),
   (43.1142838207133, -89.6543725306042),
   (43.114241167474724, -89.65403977033442)]],
 4840767: [[(43.11439198678028, -89.65767039319876),
   (43.11438637186846, -89.65702223266521),
   (43.114386444219534, -89.65663800022205),
   (43.11438500774659, -89.65642043655917)]],
 4828136: [[(43.0929398804851, -89.52993941669419),
   (43.09268999370641, -89.52972459498842),
   (43.09246213637888, -89.52964432207762),
   (43.09034734837834, -89.52964012151291)]],
 5314065: [[(43.11217899960449, -89.64643291410943),
   (43.112014039426505, -89.64616800901612),
   (43.111802073772324, -89.64582885247067)]],
 5314061: [[(43.11217899960449, -89.64643291410943),
   (43.1